In [ ]:
import numpy as n
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import files
uploaded = files.upload()

import io
mydata=pd.read_csv(io.BytesIO(uploaded['Bank_Personal_Loan_Modelling_ml.csv']))

mydata.head()

mydata.tail()

#checking missing or null values
mydata.info()

#summary of data, ie., mean, median, mode &quartile
mydata.describe()

#check howmany null value present in a column
mydata.isnull().sum()

#DATACLEANING
#DROPPING IRRELEVANT COLUMNS
#comparing similarity in age and correlation
correlation= mydata['Age'].corr(mydata['Experience'])
correlation

mydata = mydata.drop(['ID','Experience'],axis=1 )

mydata.head()

mydata.nunique()

mydata=mydata.drop(['ZIP Code'],axis=1)

mydata.head()

mydata

print('people with 0 mortrage =',len(mydata[mydata.Mortgage==0]));


print('people with 0 credit card average =',len(mydata[mydata.CCAvg==0]));


mydata.Family.value_counts()

mydata.Education.value_counts()

mydata['Securities Account'].value_counts()

mydata['CD Account'].value_counts()

mydata.CreditCard.value_counts()

mydata.Online.value_counts()

sb.pairplot(mydata)

sb.set_style(style='white')

sb.distplot(mydata.Age);

sb.distplot(mydata.Income);

sb.distplot(mydata.CCAvg);

sb.distplot(mydata.Mortgage);

sb.countplot(mydata.Family);

sb.countplot(mydata.Education);

sb.boxplot(x='Education',y='Income',hue='Personal Loan',data=mydata)

sb.countplot(x="Securities Account", data=mydata,hue="Personal Loan")

sb.countplot(x="Income", data=mydata,hue="Personal Loan")

sb.countplot(x="Age", data=mydata,hue="Personal Loan")

fig,ax=plt.subplots(figsize=(15,10))
sb.heatmap(mydata.corr(),cmap='viridis',annot=True);

DF=pd.DataFrame(mydata['Personal Loan'].value_counts()).reset_index()
DF.columns=['Labels','Personal Loan']
fig1,ax1=plt.subplots(figsize=(10,8))
explode=(0,0.15)
ax1.pie(DF['Personal Loan'],explode=explode,autopct='%1.1f%%',shadow=True,startangle=70)
ax1.axis('equal')
plt.title('Personal Loan Percentage')
plt.show()

datax=mydata.loc[:, mydata.columns != "Personal Loan"]
datay=mydata["Personal Loan"]

datax.shape,datay.shape

datax.head()

from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method="yeo-johnson",standardize=False)
pt.fit(datax["Income"].values.reshape(-1,1))
t = pt.transform(datax["Income"].values.reshape(-1,1))

sb.distplot(mydata.Income)

sb.distplot(t)

from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method="yeo-johnson",standardize=False)
pt.fit(datax["CCAvg"].values.reshape(-1,1))
t = pt.transform(datax["CCAvg"].values.reshape(-1,1))
sb.distplot(t)

datax['Mortgage_int'] = pd.cut(datax.Mortgage,
                               bins=[0,100,200,300,400,500,600,700],
                               labels=[0,1,2,3,4,5,6],
                               include_lowest=True)
datax.drop('Mortgage', axis=1, inplace=True)

datax.head()

sb.distplot(datax.Mortgage_int)


from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(datax, datay, test_size =0.3, stratify = datay, random_state=0)


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score,plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve

log_reg = LogisticRegression(C=1.0, max_iter=1000)
log_reg.fit(train_x, train_y)

class_name = ['will not take loan', 'take loan']

train_score = log_reg.score(train_x, train_y)
print('Training Accuracy:', train_score.round(2))
test_score = log_reg.score(test_x, test_y)
print('Testing Accuracy:', test_score.round(2))
#used for prediction
y_pred_logi = log_reg.predict(test_x)

precision_logi = precision_score(test_y, y_pred_logi, labels=class_names).round(2)
print('Precision:', precision_logi)
recall_logi = recall_score(test_y, y_pred_logi).round(2)
print('Recall:', recall_logi)

plot_confusion_matrix(log_reg, test_x, test_y, display_labels=class_name)
plt.title('Confusion Matrix for Logistic Regression')

plot_roc_curve(log_reg, test_x, test_y)
plt.title('ROC Curve for Logistic Regression')

plot_precision_recall_curve(log_reg, test_x, test_y)
plt.title('Precision-Recall Curve for Logistic Regression')

rfc = RandomForestClassifier(n_estimators=100, max_depth=7,n_jobs=-1, random_state=1 )
rfc.fit(train_x, train_y)

train_score = rfc.score(train_x, train_y)
print('Training Accuracy:', train_score.round(2))
test_score = rfc.score(test_x, test_y)
print('Testing Accuracy:', test_score.round(2))

y_pred_rf = rfc.predict(test_x)

precision_rf = precision_score(test_y, y_pred_rf, labels=class_names).round(2)
print('Precision:', precision_rf)
recall_rf = recall_score(test_y, y_pred_rf).round(2)
print('Recall:', recall_rf)

plot_confusion_matrix(rfc, test_x, test_y, display_labels=class_names)
plt.title('Confusion Matrix for Random Forest Classifier')

plot_roc_curve(rfc, test_x, test_y)
plt.title('ROC Curve for Random Forest Classifier')

plot_precision_recall_curve(rfc, test_x, test_y)
plt.title('Precision-Recall Random Forest Classifier')

from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(train_x, train_y)
y_pred=dtc.predict(test_x)

train_score = dtc.score(train_x, train_y)
print('Training Accuracy:', train_score.round(2))
test_score = dtc.score(test_x, test_y)
print('Testing Accuracy:', test_score.round(2))

precision_dt = precision_score(test_y, y_pred, labels=class_names).round(2)
print('Precision:', precision_dt)
recall_dt = recall_score(test_y, y_pred).round(2)
print('Recall:', recall_dt)

plot_confusion_matrix(dtc, test_x, test_y, display_labels=class_names)
plt.title('Confusion Matrix for Decision Tree Classifier')

plot_roc_curve(dtc, test_x, test_y)
plt.title('ROC Curve for Decision Tree Classifier')

plot_precision_recall_curve(dtc, test_x, test_y)
plt.title('Precision-Recall Decision Tree Classifier')

from sklearn.neighbors import KNeighborsClassifier

k= KNeighborsClassifier(n_neighbors= 15,weights ='uniform',metric='euclidean')
k.fit(train_x,train_y)
y_pred=k.predict(test_x)

train_score = k.score(train_x, train_y)
print('Training Accuracy:', train_score.round(2))
test_score = k.score(test_x, test_y)
print('Testing Accuracy:', test_score.round(2))

precision_dt = precision_score(test_y, y_pred, labels=class_names).round(2)
print('Precision:', precision_dt)
recall_dt = recall_score(test_y, y_pred).round(2)
print('Recall:', recall_dt)

plot_confusion_matrix(k, test_x, test_y, display_labels=class_names)
plt.title('Confusion Matrix for Decision Tree Classifier')

plot_roc_curve(k, test_x, test_y)
plt.title('ROC Curve for Decision Tree Classifier')

plot_precision_recall_curve(k, test_x, test_y)
plt.title('Precision-Recall Decision Tree Classifier')










